# Generation of Indian Classical Raga using Probability

Indian classical (hindustani) music has some definite structure. We will try to make use of that structure to generate indian classical tunes using a python program. Lets try to understand some basics first. Even if you ignore python programs in this article you will get good idea of how probabilities are used in this process.

### Notations
There are seven shudhha swars (or notes). All seven together are called saptak. Every next swar is a sound with frequency higher than current swar.

In [19]:
saptak = ['Sa', 'Re', "Ga", "Ma", "Pa", "Dha", "Ni"]

In addition to _shudhha_ swars , few swars have there _komal_ counter part. For Example `Re, Ga, Dha` and `Ni` these swars also come as komal in some tunes. Similarly `Ma` has tivra counter part instead of komal. `Sa` and `Pa` do not have komal or tivra versions. We will denote shudhha swar with just plain letters. Komal swar with an underscore attached to it, while for tivra swar we will add a tail of two underscore. In all we have 12 swar in one saptak then!


In [1]:
saptak = ['Sa','Re_',"Re","Ga_","Ga","Ma","Ma__","Pa","Dha_","Dha","Ni_","Ni"]
len(saptak)

12

There are three saptaks that we are going to make use of. _mandra_,_madhya_ and _tar_ saptak. We will denote all swar in _mandra_ saptak with lower case letters. Swar from _madhya_ saptak will be denoted by capitalized from (first letter upper case, rest lower), _tar_ saptak's swar by all upper case letters

In [2]:
mandra_saptak = [s.lower() for s in saptak]
mandra_saptak

['sa',
 're_',
 're',
 'ga_',
 'ga',
 'ma',
 'ma__',
 'pa',
 'dha_',
 'dha',
 'ni_',
 'ni']

In [22]:
tar_saptak = [s.upper() for s in saptak]
tar_saptak

['SA',
 'RE_',
 'RE',
 'GA_',
 'GA',
 'MA',
 'MA__',
 'PA',
 'DHA_',
 'DHA',
 'NI_',
 'NI']

### Raga ###

Raga consists of subset (not necessarily proper subset) of these 12 swar of a saptak. Further, when composing tunes in a raga, there are certain swars/notes that are allowed when notes are increasing in their frequencies (_aaroha_) and certain other notes when they are decreasing in frequency (_avroha_). Some ragas are more complicated in that when notes are increasing in frequency there are certain notes from which one must ‘descend’, i.e., use lower frequency notes and then continue the ‘ascent’ of increasing frequency notes. This last structure is also called the ‘chalan’ of the raga. Further, some notes are granted greater relative time than others, often called ‘nyas’ notes. 
For example here is a sample tune from raag bhoop.
```
SA,SA,Dha,Pa,Ga,Re,Sa,Re,Ga,Ga,Pa,Ga,Dha,Pa,Ga,Ga
Ga,Pa,Dha,SA,RE,SA,Dha,Pa,SA,Pa,Dha,Pa,Ga,Re,Sa,Sa
Ga,Ga,Pa,Dha,Pa,SA,SA,SA,Dha,Dha,SA,RE,GA,RE,SA,Dha
GA,GA,RE,SA,RE,RE,SA,Dha,SA,Pa,Dha,Pa,Ga,Re,Sa,Sa

Ga,Re,Ga,Ga,Sa,Re,Sa,Sa,Sa,Sa,Sa,dha,Sa,Re,Ga,Ga
Pa,Ga,Pa,Pa,Dha,Dha,Pa,Pa,Ga,Pa,Dha,SA,Dha,Pa,Ga,Sa
Pa,Ga,Ga,Re,Ga,Pa,SA,Dha,SA,SA,SA,SA,Dha,Re,SA,SA
Dha,Dha,Dha,Dha,SA,RE,GA,RE,SA,SA,Dha,Pa,Dha,SA,Dha,Pa
Ga,Re,Ga,Ga,Ga,Re,Pa,Ga,Dha,Pa,Dha,SA,Dha,Pa,Ga,Sa
```

### How do we learn?

We usually remember patterns in anything we learn. For example it is said that lanugages are learnt by practicing conversation. During conversations we remember that after the word `I` there are more chances of of verbs `am`, `was`, `will`, `shall` , `have`. but there are very less chances of having `is`, `were`, `has` after the word `I`. Same is true for swaras in an indian classical raga.

 ### Can we emulate the patterns of raga? ###
 
We make use of some existing tunes and assign a probabilty of transition from one swar to next. The structures/patterns defining the raga are encoded in a set of probabilities. Attached to each note is an array of probabilities which specifies the probability of transitioning from that note to other notes. In tech parlance this would be a Markov chain of sorts where the transition matrix is a stochastic matrix.
    
```
     ___0.1___ Pa        ___0.5__ Ga
    / __0.1___ Ga       /
   / / _0.3___ Re-------____0.4__ Sa
  / / /                 \
Sa _____0.2___ Sa        \__0.1__ dha
  \ 
   \ \ \_0.25_ dha 
    \ \__0.04_ pa
     \___0.01_ ga

```

### Probabilties

How do we compute the prababilty of what swar will come next to a particular swar?

So lets take our sample data and construct a transition probability for each swar! 

Here is notation of a sample tunes of raag bhoop. After every beat there is a comma. So whereever there are two or more swar without any comma in between, they are to be played in half note.

In [4]:
%%file bhoop1.csv
SA,SA,Dha,Pa,Ga,Re,Sa,Re,Ga$,Pa,Ga,Dha,Pa,Ga$
Ga,Pa,Dha,SA,RE,SA,Dha,Pa,SA,Pa,Dha,Pa,Ga,Re,Sa$
Ga,Ga,Pa,Dha,Pa,SA,SA$,Dha,Dha,SA,RE,GA,RE,SA,Dha
GA,GA,RE,SA,RE,RE,SA,Dha,SA,Pa,Dha,Pa,Ga,Re,Sa$
Ga,Re,Ga$,Sa,Re,Sa,Sa,Sa,Sa,Sa,dha,Sa,Re,Ga$
Pa,Ga,Pa,Pa,Dha,Dha,Pa,Pa,Ga,Pa,Dha,SA,Dha,Pa,Ga,Sa
Pa,Ga$,Re,Ga,Pa,SA,Dha,SA$$,SA,Dha,RE,SA$
Dha,Dha,Dha,Dha,SA,RE,GA,RE,SA,SA,Dha,Pa,Dha,SA,Dha,Pa
Ga,Re,Ga,Ga,Ga,Re,Pa,Ga,Dha,Pa,Dha,SA,Dha,Pa,Ga,Sa
Sa,Re,Ga,Pa,GaRe,Sa$,Re,Pa$$,Re,Ga$,Re$
Ga,GaPa,Ga,Re,Ga,Pa,Dha,SA,SA$,SA,Dha$,Pa,Ga,Pa
DhaRE,SA$,Dha$,Pa,Ga,Re,GaPa,DhaSA,PaDha,SA,DhaSA,DhaPa,GaRe,Sa
Pa,Ga,Ga,Ga,Pa$,SA,Dha,SA$,SA,SA,SARE,GARE,SA$
SA,Dha,Dha,SA$,SA,RE$,DhaSA,PaDha,SA$,Dha$,Pa$
Ga,GaPa,Ga,Re,Ga,Pa,Dha,SA,SARE,GARE,SA,DhaPa,DhaSA,DhaPa,GaRe,GaPa,GaRe,Sa

Overwriting bhoop1.csv


Following is some part of computer program that reads this data and processes it.

In [5]:
def histogram(data):
    """
    data is raga tune as a list of lines.
    every line is a list of swar from the tune.
    """
    hist = {}
    
    for line in data: #histogram of whats next item
        for i, item in enumerate(line[:-1]):
            itemd = hist.get(item, {})
            itemd[line[i+1]] = itemd.get(line[i+1], 0) + 1
            hist[item] = itemd

    return hist

def transition_probability(hist):
    prob = {}
    for k in hist:#compute probabilitis
        total  = sum(hist[k].values())
        prob[k] = {j: v/total for j,v in hist[k].items()}
    return prob
                
def readcsv(filename):
    with open(filename) as f:
        return [line.strip().split(",") for line in f]

def textplot(hist):
    for k in sorted(hist, key=lambda k: hist[k], reverse=False):
        print(k.rjust(9), str(hist[k]).rjust(3), hist[k]*"*")

hist_  = histogram(readcsv("bhoop1.csv"))

We will use a computer program to read above notations from a file. We will compute simple frequencies, for each swar how many times it occurred after a particular swar. We will call this data as histogram for a particular swar.

Lets look at what does this histogram means! For example look at following text chart. If we look at histogram data of `Dha`, it gives a glimpse of which swar comes after `Dha` and how many times.

In [28]:
textplot(hist_['Dha'])

     SA$$   1 *
       RE   1 *
      SA$   2 **
      Dha   6 ******
       SA   9 *********
       Pa  12 ************


This means in our training data, `Pa` has occurred more than other note after `Dha`. So next most probable note after `Dha` is `Pa` while next to that counts highest after `Pa` is `SA`.

And now probability (chance) that `Pa` will come after `Dha` is `12/(1+1+2+6+9+12) = 0.387`. Similarly we compute probabilities for every possible occurrence of other swar. Note that if some swar is not there in histogram or transistion probabilities of a particular sawr means it has zero probability or it will never follow the particular swar.

Probabilitis for `Dha` look like as given below.

In [29]:
probs = transition_probability(hist_)
probs['Dha']

{'Pa': 0.3870967741935484,
 'SA': 0.2903225806451613,
 'Dha': 0.1935483870967742,
 'SA$$': 0.03225806451612903,
 'RE': 0.03225806451612903,
 'SA$': 0.06451612903225806}

This means 

probability of `Pa` coming after `Dha` is 0.3870967741935484

probability of `SA` coming after `Dha` is 0.2903225806451613

probability of `Dha` coming after `Dha` is 0.1935483870967742,

probability of `SA$$` coming after `Dha` is 0.03225806451612903,

probability of `RE` coming after `Dha` is 0.03225806451612903,

probability of  `SA$` coming after `Dha` is 0.06451612903225806


### Sampling , given the probabilities

Now our next problem is given these transition probabilities for `Dha` how do we sample so that we find next note based on probabilty, but same time random also! This problem boils down to a problem of sampling item from an array based on probability for each position.


In [30]:
import random
def sample(items, probs):
    r = random.random()
    index = 0
    while(r >= 0 and index < len(probs)):
        r -= probs[index]
        index += 1
    return items[index - 1]


def test_sample():
    probs = {'Pa': 0.40425531914893614,
             'SA': 0.3617021276595745,
             'Dha': 0.19148936170212766,
             'Re': 0.02127659574468085,
             'Ga': 0.02127659574468085}
    keys = [i for i in probs.keys()]
    pvalues = [probs[i] for i in keys]
    samples = [sample(keys, pvalues) for i in range(10000)]
    for item in probs:
        print(item, samples.count(item)/10000)
    assert abs(probs['Pa']-samples.count('Pa')/10000)<0.01
    
test_sample()


Pa 0.4124
SA 0.3656
Dha 0.1804
Re 0.0209
Ga 0.0207


### lets ask computer to generate a tune!
Once we have such transition probabilities of every swar in that raga, then if starting note is given we can generate next note based on prababilities given by our model. Then again the generated note used as starting point and again what can come next to it is again predicted. If we contnue same logic , we can keep generating notes after note and if we play those notes it will surprise us. It will be a different tune but in same raga in which training was done.

Lets create a sample random tune in raag bhoop!

In [31]:
bhoop_probs = transition_probability(histogram(readcsv("bhoop1.csv")))
def aalap(initial, transition_prob):
    current = initial
    
    while True:
        if current not in transition_prob:
            current = random.choice(list(transition_prob.keys()))
        yield current
        probs = transition_prob[current]
        items = [i for i in probs.keys()]
        pvalues = [probs[item] for item in items]
        current = sample(items, pvalues)
            
def take(seq, n):
    return [next(seq) for i in range(n)]


In [32]:
bhoop = aalap("Sa",bhoop_probs)

In [33]:
take(bhoop, 32)

['Sa',
 'Sa',
 'Sa',
 'Re',
 'Ga',
 'Pa',
 'Ga',
 'Pa',
 'Pa',
 'Dha',
 'Pa',
 'SA',
 'SA',
 'Dha',
 'Pa',
 'Dha',
 'Pa',
 'Ga',
 'Dha',
 'RE',
 'GA',
 'GA',
 'RE',
 'SA',
 'Dha',
 'Dha',
 'Dha',
 'SA',
 'RE',
 'SA',
 'Dha',
 'SA$$']

### How do we examine it!!
one way is to play it or sing it! Masters of classical music will imdiately recognise if it is the same raga. But there is another way! When we listen to a tune, how do we understand that it is playing in raag `bhoop`? By special sequencs of notes that come frequently in raag bhoop. we call it chalan or pakad. So let our python function generate some long tune for sufficiently long time. we will count how many times pakad comes into that sequence. This is just one indicator. Note that pakad has emerged into the sequence genereated by the computer program. We have not given any prior input about what pakad is there in the model that we built.

In [34]:
from collections import deque

def search(seq, subseq, end=100):
    def compare(source, dest):
        for item in dest:
            return any(["".join(item).lower() in "".join(source).lower().replace("$","") for item in dest])
    
    n = len(max(subseq, key=len))
    window = deque(take(seq, n), n)
    for i in range(n, end):
        if compare(window, subseq):
            yield i-n
            window = deque(take(seq, n), n)
        else:
            window.append(next(seq))
            

def count(seq):
    return sum(1 for i in seq)
        

In [35]:
bhoop = aalap("Sa", bhoop_probs)
pakad = [["dha","dha","sa"],["ga","re","pa","ga"],["dha","pa","ga","re"]]
print("Notes", "Pakad")
for j in range(8,100,8):
    print("{:5} {:5}".format(j,sum([count(search(bhoop,pakad, j)) for i in range(1000)])/1000))

Notes Pakad
    8 0.161
   16 0.423
   24 0.673
   32 0.958
   40 1.261
   48 1.579
   56 1.775
   64 2.075
   72 2.394
   80 2.626
   88 2.951
   96  3.19
